In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [2]:
import os
if not os.path.isdir("Conditional-Gan"):
  os.makedirs("Conditional-Gan")
%cd Conditional-Gan

/content/drive/MyDrive/Conditional-Gan


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D,Dropout,Input,UpSampling2D,BatchNormalization,Flatten,Dense,Reshape,Embedding,concatenate,MaxPooling2D
from tensorflow.keras.utils import plot_model

In [4]:
def discriminator(input_shape,n_classes=10):
  inp_img=Input(shape=input_shape ,name="image")
  inp_label=Input(shape=(1,),name="label")
  emb=Embedding(n_classes,50)(inp_label)
  il=Dense(28*28,activation="relu")(emb)
  il=Reshape((28,28,1))(il)


  con=concatenate([inp_img,il])
  x=Conv2D(64, 3, activation = 'relu')(con)
  x=MaxPooling2D((2,2))(x)
  x=Conv2D(64, 3, activation= 'relu')(x)
  x=MaxPooling2D((2,2))(x)

  x=Flatten()(x)
  x=Dense(32,activation='relu')(x)
  output=Dense(1,activation="sigmoid")(x)
  model = Model(inputs=[inp_img,inp_label],outputs=output,name="discrminator")
  model.summary()
  model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=2e-4), metrics=['accuracy'])
  return model

In [5]:
D_model=discriminator((28,28,1),n_classes=10)

Model: "discrminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 label (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 50)        500         ['label[0][0]']                  
                                                                                                  
 dense (Dense)                  (None, 1, 784)       39984       ['embedding[0][0]']              
                                                                                                  
 image (InputLayer)             [(None, 28, 28, 1)]  0           []                               
                                                                                       

In [6]:
def Generator(latent_dim,n_classes):

  in_label=Input(shape=(1,),name="input_label")
  emb=Embedding(n_classes,50)(in_label)
  n=7*7*128
  de=Dense(n,activation='relu')(emb)
  reshape=Reshape((7,7,128))(de)

  in_lat=Input(shape=(latent_dim,),name="latent_input")
  nod=7*7*128
  lat=Dense(nod,activation='leaky_relu')(in_lat)
  lat=Reshape((7,7,128))(lat)
  x=concatenate([lat,reshape])
  x=UpSampling2D((2,2))(x)
  x=Conv2D(128,3,padding='same',activation='leaky_relu')(x)
  x=UpSampling2D((2,2))(x)
  x=Conv2D(64,3,padding='same',activation='leaky_relu')(x)
  output=Conv2D(1,3,padding='same',activation='sigmoid')(x)
  model=Model([in_lat,in_label],output,name='Generator')
  model.summary()
  return model

In [ ]:
G_model=Generator(100,n_classes=10)

Model: "Generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_label (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 latent_input (InputLayer)      [(None, 100)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 50)        500         ['input_label[0][0]']            
                                                                                                  
 dense_4 (Dense)                (None, 6272)         633472      ['latent_input[0][0]']           
                                                                                          

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(learning_rate=2e-4)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
gan=define_gan(G_model,D_model)

In [ ]:
# load fashion mnist images
import tensorflow as tf
import numpy as np

def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) =  tf.keras.datasets.fashion_mnist.load_data()
	# expand to 3d, e.g. add channels
	X = np.expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = X/255.
	return [X, trainy]

# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = np.random.randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = tf.ones((n_samples, 1))
	return [X, labels], y

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = np.random.randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input],verbose=0)
	# create class labels
	y = np.zeros((n_samples, 1))
	return [images, labels_input], y

In [ ]:
from keras.utils import Progbar

def train(g_model, d_model, gan_model, dataset, latent_dim, epochs=100, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # Manually enumerate epochs
    for epoch in range(1,epochs+1):
        print(f"Epoch {epoch}/{epochs}")
        # Initialize the progress bar
        progbar = Progbar(bat_per_epo)
        
        # Enumerate batches over the training set
        for j in range(bat_per_epo):
            # Get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            
            # Update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
            
            # Generate 'fake' examples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            
            # Update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
            
            # Prepare points in latent space as input for the generator
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            
            # Create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            
            # Update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
            
            # Update the progress bar with the current batch metrics
            progbar.update(j + 1, values=[("d1", d_loss1), ("d2", d_loss2), ("g", g_loss)])
        # Save the generator model
        g_model.save('generator.hdf5')
        d_model.save('Discriminator.hdf5')


In [ ]:
# size of the latent space
latent_dim = 100
# load image data
dataset = load_real_samples()
# train model
train(G_model, D_model, gan, dataset, latent_dim, epochs=500)